# "1.4- NLP: Autocorrector"
> "Procesamiento del lenguaje natural con Modelos Probabilísticos"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, NLP, ML]
- image: images/framework.jpg
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

Fuentes:


[DeepLearning.AI: Procesamiento de lenguaje natural. Programa especializado en Coursera](https://www.coursera.org/specializations/natural-language-processing) 


[How to Write a Spelling Corrector, Peter Norvig, 2007](https://norvig.com/spell-correct.html)

> tip: Puedes ver este post en GitHub o ejecutarlo en Binder o Google Colab, pulsa el icono.

# Construir el modelo

## Creación del vocabulario

In [25]:
# imports
import re # regular expression library; for tokenization of words
from collections import Counter # collections library; counter: dict subclass for counting hashable objects
import matplotlib.pyplot as plt # for data visualization
import numpy as np
import pandas as pd

In [2]:
# the tiny corpus of text ! 
text = 'red pink pink blue blue yellow ORANGE BLUE BLUE PINK' # 🌈
print(text)
print('string length : ',len(text))

red pink pink blue blue yellow ORANGE BLUE BLUE PINK
string length :  52


### Preprocesamiento

In [26]:
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You just have to read it in. 
    Output: 
        words: a list containing all the words in the corpus (text file you read) in lower case. 
    """
    words = [] # return this variable correctly
   
    #Open the file, read its contents into a string variable
    with open(file_name, 'r') as file:
        text = file.read()
    # convert all letters to lower case
    text_lowercase = text.lower()
    #Convert every word to lower case and return them in a list.
    words = re.findall(r'\w+', text_lowercase)
    
    return words

In [28]:
word_l = process_data('./data/shakespeare.txt')
vocab = set(word_l)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


### Creación del vocabulario
Un conjunto de palabras distintas del texto.

In [9]:
# create vocab
vocab = set(words)
print(vocab)
print('count : ',len(vocab))

{'red', 'blue', 'pink', 'orange', 'yellow'}
count :  5


### Agregar información con el conteo de palabras

In [12]:
# Opción 1: crear vocabulario incluyendo el conteo de palabras
counts_a = dict()
for w in words:
    counts_a[w] = counts_a.get(w,0)+1
print(counts_a)
print('count : ',len(counts_a))

{'red': 1, 'pink': 3, 'blue': 4, 'yellow': 1, 'orange': 1}
count :  5


In [13]:
# Opción 2: crear vocabulario usando collections.Counter
counts_b = dict()
counts_b = Counter(words)
print(counts_b)
print('count : ',len(counts_b))

Counter({'blue': 4, 'pink': 3, 'red': 1, 'yellow': 1, 'orange': 1})
count :  5


In [36]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''
    word_count_dict = {}  # fill this with word counts
    word_count_dict = Counter(word_l)   

    return word_count_dict

In [37]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")

There are 6116 key values pairs
The count for the word 'thee' is 240


## Identificar una palabra mal escrita
Se puede comporbar si está en el vocabulario. Si no lo está puede ser un candidato.

## Encontrar cadenas que están a n ediciones de distancia
Estas podrían ser cadenas aleatorias.

Editar: operación en una cadena para cambiarla.

- Operaciones:
  - Insertar: añadir una letra
  - Borrar: eleminar una letra
  - Cambiar: intercambiar dos letras seguidas
  - Sustituir: cambiar una letra por otra

In [15]:
# data
word = 'dearz'

### Divisiones
Encontrar todas las formas en que se puede dividir la palabra en 2 partes

In [17]:
# Opción 1: con un bucle
splits_a = []
for i in range(len(word)+1):
    splits_a.append([word[:i],word[i:]])

for i in splits_a:
    print(i)

['', 'dearz']
['d', 'earz']
['de', 'arz']
['dea', 'rz']
['dear', 'z']
['dearz', '']


In [18]:
# Opción 2: con list comprehension
splits_b = [(word[:i], word[i:]) for i in range(len(word) + 1)]

for i in splits_b:
    print(i)

('', 'dearz')
('d', 'earz')
('de', 'arz')
('dea', 'rz')
('dear', 'z')
('dearz', '')


### Operación borrar

In [20]:
# Opción 1: con un bucle
splits = splits_a
deletes = []

print('word : ', word)
for L,R in splits:
    if R:
        print(L + R[1:], ' <-- delete ', R[0])

word :  dearz
earz  <-- delete  d
darz  <-- delete  e
derz  <-- delete  a
deaz  <-- delete  r
dear  <-- delete  z


In [21]:
# Opción 2: con list comprehension
splits = splits_a
deletes = [L + R[1:] for L, R in splits if R]

print(deletes)
print('*** which is the same as ***')
for i in deletes:
    print(i)

['earz', 'darz', 'derz', 'deaz', 'dear']
*** which is the same as ***
earz
darz
derz
deaz
dear


In [40]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    delete_l = []
    split_l = []
    split_l = [ (word[:i],word[i:]) for i in range(len(word))]
    delete_l= [ L+R[1:] for L,R in split_l if R]

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return  delete_l

In [46]:
delete_word_l = delete_letter(word="Soleado",
                        verbose=True)

input word Soleado, 
split_l = [('', 'Soleado'), ('S', 'oleado'), ('So', 'leado'), ('Sol', 'eado'), ('Sole', 'ado'), ('Solea', 'do'), ('Solead', 'o')], 
delete_l = ['oleado', 'Sleado', 'Soeado', 'Solado', 'Soledo', 'Soleao', 'Solead']


### Operación cambiar

In [47]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    switch_l = []
    split_l = []
    split_l = [ (word[:i],word[i:]) for i in range(len(word))]
    switch_l = [ L+R[:2][::-1]+R[2:] for L,R in split_l if len(R)>1]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 
    
    return switch_l

In [48]:
switch_word_l = switch_letter(word="soleado",
                         verbose=True)

Input word = soleado 
split_l = [('', 'soleado'), ('s', 'oleado'), ('so', 'leado'), ('sol', 'eado'), ('sole', 'ado'), ('solea', 'do'), ('solead', 'o')] 
switch_l = ['osleado', 'sloeado', 'soelado', 'solaedo', 'soledao', 'soleaod']


### Operación sustituir

In [49]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    
    replace_l = []
    split_l = []
    split_l = [ (word[:i],word[i:]) for i in range(len(word))]
    replace_l = [L+c+R[1:] for L,R in split_l if len(R)>0 for c in letters]
    replace_set=set(replace_l)
    replace_set.discard(word)
   
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [50]:
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


### Operación insertar

In [51]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    split_l = [ (word[:i],word[i:]) for i in range(len(word)+1)]  
    insert_l = [L+c+R for L,R in split_l if len(R)>-1 for c in letters]
    
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [52]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


### Función editar una letra
Para obtener todas las ediciones posibles que están a una edición de distancia de una palabra

In [53]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """   
    edit_one_set = set()
    edit_one_set = delete_letter(word)+ replace_letter(word)+insert_letter(word)
    
    if allow_switches:
        edit_one_set += switch_letter(word)    

    return set(edit_one_set)

In [54]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


### Función editar dos letras
Para obtener todas las ediciones posibles que están a dos ediciones de distancia de una palabra

In [55]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''    
    edit_two_set = set()
    
    for w in edit_one_letter(word,allow_switches):
        edit_two_set.update(edit_one_letter(w,allow_switches))  
   
    # return this as a set instead of a list
    return set(edit_two_set)

In [57]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


## Filtrar candidatos
Mantener solo las palabras reales de los pasos anteriores

In [23]:
vocab = ['dean','deer','dear','fries','and','coke']
edits = list(deletes)

print('vocab : ', vocab)
print('edits : ', edits)

candidates=[]
candidates = set(edits).intersection(vocab)  # hint: 'set.intersection'


print('candidate words : ', candidates)

vocab :  ['dean', 'deer', 'dear', 'fries', 'and', 'coke']
edits :  ['earz', 'darz', 'derz', 'deaz', 'dear']
candidate words :  {'dear'}


## Calcular probabilidades de palabras
Elegir la palabra que es más probable que ocurra en ese contexto.

![](images/nlp036.png)

In [38]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {}  # return this variable correctly
    
    # get the total count of words for all words in the dictionary
    M = sum(word_count_dict.values())    
    
    #for k in word_count_dict:
     #   probs[k]=word_count_dict[k]/M
    
    probs = { k:(v/M) for (k,v) in word_count_dict.items()}
    

    return probs

In [39]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


## Distancia mínima de edición

Permite:

- Evaluar la similitud entre dos cadenas
- Encontrar el número mínimo de ediciones entre dos cadenas
- Implementar un corrección ortográfica, similitud de documentos, traducción automática, secuenciación de ADN y más

Coste = distancia de edición entre dos cadenas.

![](images/nlp037.png)

### Algoritmo de distancia mínima de edición
Se comienza con una palabra de origen y la transforma en la palabra de destino. 

Para pasar de # -> #  se necesita un costo de 0.
De p -> # se necesita un costo de 1, costo de una eliminación.
De p -> s es 2,  costo mínimo que uno podría usar para llegar de p a s.

Seguir así completando un elemento a la vez. 


![](images/nlp038.png)
![](images/nlp039.png)
![](images/nlp040.png)

Esta es la distancia de Levenshtein que especifica el costo por operación.
Si se necesita reconstruir la ruta de cómo llegó de una cadena a la otra, puede usar el camino inverso.

Esta es la técnica de **Programación dinámica**:  Primero resuelve el subproblema más pequeño primero y luego, reutilizando ese resultado, resuelve el siguiente subproblema más grande, guardando ese resultado, reutilizándolo nuevamente, y así sucesivamente. Esto es exactamente lo que hizo al completar cada celda desde la esquina superior derecha hasta la esquina inferior izquierda. 